In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [0]:
!pip install tf-nightly

In [0]:
!pip install tensorflow==1.14

In [0]:
!pip uninstall tensorflow

In [0]:
!pip list | grep tensorflow

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# !pip install scikit-image

In [0]:
import cv2
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import skimage as sk
from random import shuffle
from tqdm import tqdm
from skimage import morphology, filters
from keras import backend as k
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization
from keras.models import Model, Sequential, load_model
from sklearn.model_selection import train_test_split
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
IMG_SIZE = 32
# how many samples to calculte the error
BATCH_SIZE = 32
TRAIN_DIR = "/content/drive/My Drive/Data Set Traffice Signs detection/train_data.npy"
TEST_DIR = "/content/drive/My Drive/Data Set Traffice Signs detection/test_data.npy"
classes = ["0","1","2","3","4","5",
           "6","7","8","9","10","11",
           "12","13","14","15","16","17",
           "18","19","20","21","22","23",
           "24","25","26","27","28","29",
           "30","31","32","33","34","35",
           "36","37","38","39","40","41","42"
           ]
           
def histogram_equalize(image):
    kernel =  sk.morphology.disk(30)
    img_local = sk.filters.rank.equalize(image, selem=kernel)
    return img_local

def create_label(folder_name):
    """ Create an one-hot encoded vector from folder name """
    index = classes.index(folder_name)
    return np.array([1 if i == index else 0 for i in range(43)])

def create_train_data():
    training_data = []
    for folder in tqdm(classes):
        folder_path = TRAIN_DIR + "/" + folder
        for train_img in os.listdir(folder_path):
            img_path = os.path.join(folder_path, train_img)
            train_img_data = cv2.imread(img_path, 0)
            train_img_data = histogram_equalize(train_img_data)
            train_img_data = cv2.resize(train_img_data, (IMG_SIZE, IMG_SIZE)) / 255.0
            training_data.append([np.array(train_img_data), create_label(folder)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def create_test_data():
    testing_data = []
    for test_img in tqdm(os.listdir(TEST_DIR)):
        img_path = os.path.join(TEST_DIR, test_img)
        if test_img == "GT-final_test.csv":
          continue
        test_img_data = cv2.imread(img_path, 0)
        test_img_data = histogram_equalize(test_img_data)
        test_img_data = cv2.resize(test_img_data, (IMG_SIZE, IMG_SIZE)) / 255.0
        testing_data.append([np.array(test_img_data), test_img])
    np.save('test_data.npy', testing_data)
    return testing_data
train_data_npy = '/content/drive/My Drive/Data Set Traffice Signs detection/train_data.npy'
test_data_npy = '/content/drive/My Drive/Data Set Traffice Signs detection/test_data.npy'
if os.path.exists(train_data_npy):
    train_data_from_file = np.load(train_data_npy, allow_pickle=True)
    print("Train Data Loaded Successfully")
    print('--------------------------------')
else:
    train_data_from_file = create_train_data()

if os.path.exists(test_data_npy):
    test_data_from_file = np.load(test_data_npy, allow_pickle=True)
    print("Teset Data Loaded Successfully")
    print('--------------------------------')
else:
    test_data_from_file = create_test_data()

train = train_data_from_file
test = test_data_from_file

X_train = np.array([i[0] for i in train]).reshape((-1, IMG_SIZE, IMG_SIZE, 1))
y_train = [i[1] for i in train]
y_train = np.asarray(y_train)

X_test = np.array([i[0] for i in test]).reshape((-1, IMG_SIZE, IMG_SIZE, 1))
y_test = [i[1] for i in test]
y_test = np.asarray(y_test)

# Model
# k.clear_session()
model = Sequential()
# model.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
model.add(Conv2D(input_shape=(IMG_SIZE,IMG_SIZE,1),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), padding="VALID"))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), padding="VALID"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2), padding="VALID"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))

model.add(Flatten())
model.add(Dense(units=256,activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(units=43, activation="softmax"))

if os.path.exists("/content/model.h5"):
  with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    model = load_model("/content/model.h5")
  print("Loaded model from disk")
  print('--------------------------------')
else:
  # save the model
  model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=Adam(lr=0.001))
  model.fit(x=X_train, y=y_train, epochs=15, validation_split=0.11, batch_size=64)
  model.save("model.h5")
  print("Saved model to disk")
  print('--------------------------------')

# testing
fields = ["Path", "ClassId"]
data = pd.read_csv("/content/drive/My Drive/Data Set Traffice Signs detection/Test.csv",  usecols=fields)
test_original = data.to_numpy()
row , col = test_original.shape
accepted_prediction = 0
for img_counter in range(len(X_test)):
  test_img_name = 'Test/' + y_test[img_counter]
  x_test_4d= np.expand_dims(X_test[img_counter], axis=0)
  prediction = model.predict(x_test_4d)
  for i in range(row):
    if test_original[i][1] == test_img_name:
      class_value = test_original[i][0]
      break
  if(np.argmax(prediction) == class_value):
    accepted_prediction += 1
print("final acc is {}".format((accepted_prediction/row) * 100))



Using TensorFlow backend.


Train Data Loaded Successfully
--------------------------------
Teset Data Loaded Successfully
--------------------------------


RuntimeError: ignored